In [218]:
import utils

import re
from itertools import product

## Day 4: Ceres Search

[#](https://adventofcode.com/2024/day/4) We have a grid of letters, on which we need to find how many times the word **XMAS** appears. Looking at the sample, it looks like letters can be reused.

In [11]:
sample_input: str = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

puzzle_input = utils.get_input(4, splitlines=False)

In [192]:
def make_grid(input_str=sample_input):
    """returns 2d grid"""
    return [[char for char in row] for row in input_str.strip().splitlines()]


def parse_input(input_str=sample_input, debug: bool = False):
    """returns grid, assumed here had to get rid of non XMAS chars"""
    data = []
    for i, line in enumerate(input_str.splitlines()):
        l = [c if c in "XMAS" else "." for c in line]
        if debug:
            print(f"Line {i}: '{line}' | : {l}")
        data.append(l)
    return data


grid = parse_input(sample_input, True)

Line 0: 'MMMSXXMASM' | : ['M', 'M', 'M', 'S', 'X', 'X', 'M', 'A', 'S', 'M']
Line 1: 'MSAMXMSMSA' | : ['M', 'S', 'A', 'M', 'X', 'M', 'S', 'M', 'S', 'A']
Line 2: 'AMXSXMAAMM' | : ['A', 'M', 'X', 'S', 'X', 'M', 'A', 'A', 'M', 'M']
Line 3: 'MSAMASMSMX' | : ['M', 'S', 'A', 'M', 'A', 'S', 'M', 'S', 'M', 'X']
Line 4: 'XMASAMXAMM' | : ['X', 'M', 'A', 'S', 'A', 'M', 'X', 'A', 'M', 'M']
Line 5: 'XXAMMXXAMA' | : ['X', 'X', 'A', 'M', 'M', 'X', 'X', 'A', 'M', 'A']
Line 6: 'SMSMSASXSS' | : ['S', 'M', 'S', 'M', 'S', 'A', 'S', 'X', 'S', 'S']
Line 7: 'SAXAMASAAA' | : ['S', 'A', 'X', 'A', 'M', 'A', 'S', 'A', 'A', 'A']
Line 8: 'MAMMMXMMMM' | : ['M', 'A', 'M', 'M', 'M', 'X', 'M', 'M', 'M', 'M']
Line 9: 'MXMXAXMASX' | : ['M', 'X', 'M', 'X', 'A', 'X', 'M', 'A', 'S', 'X']


First up, getting a list of points where X appears. This is not really needed, but lowers the checks for Xmas later on. This uses re, though I could have enumerated through each row instead.

In [112]:
def get_char_positions(input_str=sample_input, char="X"):
    Xs = []
    for y, row in enumerate(input_str.splitlines()):
        for x in (match.start() for match in re.finditer(char, row)):
            Xs.append((x, y))
    return Xs


Xs = get_char_positions()
f"first few positions of X: {Xs[:6]}"

'first few positions of X: [(4, 0), (5, 0), (4, 1), (2, 2), (4, 2), (9, 3)]'

In [ ]:
def check_x(pos: tuple[int, int], grid=grid, debug=False):
    """returns count of XMAS for a position"""
    X, Y = len(grid[0]), len(grid)
    x, y = pos
    count = 0

    if not grid[y][x] == "X":
        return 0

    directions = [
        (0, 1),  # Right →
        (0, -1),  # Left ←
        (1, 0),  # Down ↓
        (-1, 0),  # Up ↑
        (1, 1),  # Diagonal Down-Right ↘
        (-1, -1),  # Diagonal Up-Left ↖
        (1, -1),  # Diagonal Down-Left ↙
        (-1, 1),  # Diagonal Up-Right ↗
    ]

    for dx, dy in directions:
        if 0 <= (x + dx * 3) < X and 0 <= (y + dy * 3) < Y:  # check on grid
            if (
                (grid[y + dy][x + dx] == "M")
                and (grid[y + dy * 2][x + dx * 2] == "A")
                and (grid[y + dy * 3][x + dx * 3] == "S")
            ):
                count += 1

    if debug:
        print(f"{pos=} {count=}")
    return count


check_x((9, 9), debug=True)

pos=(9, 9) count=2


2

In [208]:
def solve(inp: str = sample_input, debug: bool = False):
    Xs = get_char_positions(inp)
    grid = parse_input(inp)

    count = [check_x(pos, grid) for pos in Xs]
    ans = sum(count)

    if debug:
        print(f"Number of Xs: {len(Xs)}")
        print(f"{ans=} {count=}")

    return {"result": ans, "count": count}


assert solve(sample_input, True)["result"] == 18  # sample ans check
results = solve(puzzle_input, debug=False)
print(f"\nPart 1: {results["result"]}")

Number of Xs: 19
ans=18 count=[1, 1, 1, 0, 0, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 3, 2]

Part 1: 2434


## Part 2 

Trickier! Check that two MAS form an X:

```
M.S
.A.
M.S
```

So now we look for every A and see if it forms the center of an X-MAS. My code assumes letters can be reused, e.g an M could be forming a MAS for two different As.. otherwise this solution would have to track all positions used.

Since the words can be MAS, SAM or any combination therof, lets get all possible combinations using [itertools.product](https://docs.python.org/3/library/itertools.html#itertools.product).

In [219]:
combinations = tuple(product(("MAS", "SAM"), repeat=2))
combinations

(('MAS', 'MAS'), ('MAS', 'SAM'), ('SAM', 'MAS'), ('SAM', 'SAM'))

That was only 4 combinations! I could have typed that out, but even getting to 

In [220]:
def check_mas(pos: tuple[int, int], grid=grid, debug=False) -> int:
    """returns 1 if an A forms a MAS, 0 otherwise"""
    X, Y = len(grid[0]), len(grid)
    x, y = pos
    char = grid[y][x]

    assert char == "A"

    for mas1, mas2 in combinations:
        if 0 <= x - 1 < X and 0 <= x + 1 < X and 0 <= y - 1 < Y and 0 <= y + 1 < Y:
            if (
                (grid[y - 1][x - 1] == mas1[0])  # top left
                and (grid[y + 1][x + 1] == mas1[2])  # bottom right
                and (grid[y - 1][x + 1] == mas2[0])  # top right
                and (grid[y + 1][x - 1] == mas2[2])  # bottom left
            ):
                if debug:
                    print(f"{pos=} forms an X-MAS")
                return True
    return False


As = get_char_positions(sample_input, char="A")
grid = parse_input(sample_input)
assert sum([check_mas(pos, debug=False) for pos in As]) == 9

In [221]:
def solve_2(inp: str = sample_input, debug: bool = False):
    grid = parse_input(inp)
    As = get_char_positions(inp, char="A")

    ans = sum([check_mas(pos, grid, debug) for pos in As])

    return {"result": ans}


results = solve_2(puzzle_input, debug=False)
print(f"Part 2: {results["result"]}")

Part 2: 1835
